## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.38,74,9,7.94,clear sky
1,1,Port Alfred,ZA,-33.5906,26.8910,58.87,76,54,5.12,broken clouds
2,2,Tombouctou,ML,20.0000,-3.0000,92.75,7,1,8.39,clear sky
3,3,Nalut,LY,30.3333,10.8500,71.65,10,0,11.83,clear sky
4,4,Kodiak,US,57.7900,-152.4072,45.91,76,100,27.63,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 77


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
perfect_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
perfect_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.38,74,9,7.94,clear sky
3,3,Nalut,LY,30.3333,10.8500,71.65,10,0,11.83,clear sky
7,7,Cascais,PT,38.6979,-9.4215,66.97,76,75,25.01,broken clouds
18,18,San Juan,AR,-31.5375,-68.5364,65.93,24,0,0.54,clear sky
19,19,Gold Coast,AU,-28.0000,153.4333,64.99,97,100,9.22,moderate rain
20,20,Mahebourg,MU,-20.4081,57.7000,73.69,73,20,8.05,few clouds
22,22,Ponta Do Sol,PT,32.6667,-17.1000,68.00,84,24,2.44,few clouds
24,24,Dzaoudzi,YT,-12.7887,45.2699,76.89,94,0,5.75,clear sky
29,29,Riviere-Au-Renard,CA,48.9957,-64.3971,70.09,82,100,12.35,overcast clouds
31,31,Juba,SS,4.8517,31.5825,72.27,85,76,4.25,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.
perfect_cities_df.notnull().sum()

City_ID                266
City                   266
Country                266
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Current Description    266
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = perfect_cities_df.dropna()
clean_df.count()

City_ID                266
City                   266
Country                266
Lat                    266
Lng                    266
Max Temp               266
Humidity               266
Cloudiness             266
Wind Speed             266
Current Description    266
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.38,clear sky,-23.1203,-134.9692,
3,Nalut,LY,71.65,clear sky,30.3333,10.8500,
7,Cascais,PT,66.97,broken clouds,38.6979,-9.4215,
18,San Juan,AR,65.93,clear sky,-31.5375,-68.5364,
19,Gold Coast,AU,64.99,moderate rain,-28.0000,153.4333,
20,Mahebourg,MU,73.69,few clouds,-20.4081,57.7000,
22,Ponta Do Sol,PT,68.00,few clouds,32.6667,-17.1000,
24,Dzaoudzi,YT,76.89,clear sky,-12.7887,45.2699,
29,Riviere-Au-Renard,CA,70.09,overcast clouds,48.9957,-64.3971,
31,Juba,SS,72.27,broken clouds,4.8517,31.5825,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.38,clear sky,-23.1203,-134.9692,People ThankYou
3,Nalut,LY,71.65,clear sky,30.3333,10.8500,
7,Cascais,PT,66.97,broken clouds,38.6979,-9.4215,Hotel Vila Galé Cascais
18,San Juan,AR,65.93,clear sky,-31.5375,-68.5364,Hotel Provincial
19,Gold Coast,AU,64.99,moderate rain,-28.0000,153.4333,Mantra Towers of Chevron Surfers Paradise
20,Mahebourg,MU,73.69,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
22,Ponta Do Sol,PT,68.00,few clouds,32.6667,-17.1000,Hotel do Campo
24,Dzaoudzi,YT,76.89,clear sky,-12.7887,45.2699,Le Rocher
29,Riviere-Au-Renard,CA,70.09,overcast clouds,48.9957,-64.3971,Auberge Le Caribou
31,Juba,SS,72.27,broken clouds,4.8517,31.5825,ROYAL PALACE HOTEL - JUBA


In [16]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd></dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))